<a href="https://colab.research.google.com/github/1405043-kd/QuoraInsincereQuestionClassification/blob/master/QuoraTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
trainDf=pd.read_csv("/content/gdrive/My Drive/colabData/MlProject/train.csv")

In [0]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

--2019-02-03 12:53:13--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2019-02-03 12:53:13--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  9.86MB/s    in 78s     

2019-02-03 12:54:31 (26.6 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [0]:
import zipfile
zipRef=zipfile.ZipFile('glove.840B.300d.zip','r')
zipRef.extractall()
zipRef.close()

In [0]:
import numpy as np 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [0]:
def readEmbeddings(embeddingsFile):
    tempDict={}
    f=open(embeddingsFile)
    for line in tqdm(f):
        values=line.split(" ")
        word=values[0]
        vect=np.asarray(values[1:],dtype='float32')
        tempDict[word]=vect
    f.close()
    return tempDict
embeddingsDict=readEmbeddings('glove.840B.300d.txt')

vectDim=300
numOfVect=30
len(embeddingsDict['the'])

2196017it [02:57, 12358.68it/s]


300

In [0]:
def textToArr(txt):
    zeroVect=np.zeros(vectDim)
    txt=txt[:-1].split()[:numOfVect]
    embeds=[embeddingsDict.get(x,zeroVect) for x in txt]
    embeds+=[zeroVect]*(numOfVect-len(embeds)) 
    return np.array(embeds)

textToArr("what is quora?").shape

(30, 300)

In [0]:
trainDf, valDf = train_test_split(trainDf, test_size=0.4)
valDf, testDf = train_test_split(valDf, test_size=0.5)
print(trainDf.shape)
print(testDf.shape)
print(valDf.shape)
testDf.head(3)

(783673, 3)
(261225, 3)
(261224, 3)


,qid,question_text,target
1087474,d5192caf5c097bea6050,Where are the best websites for free advertising?,0
404075,4f2ed12f534457180a63,What challenges did V.S. Naipaul face when bec...,0
230531,2d157916e200e201b407,How does one travel on a US fake passport? Do ...,0


In [0]:
valVects = np.array([textToArr(x) for x in tqdm(valDf["question_text"])])
valY = np.array(valDf["target"])

valVects[0].shape

 44%|████▍     | 115805/261224 [01:05<62:30:50,  1.55s/it]

In [0]:
testVects = np.array([textToArr(x) for x in tqdm(testDf["question_text"])])
testY = np.array(testDf["target"])

100%|██████████| 104490/104490 [00:07<00:00, 13250.66it/s]


In [0]:
batchSize = 128

def batchGenerate(train_df):
    numBatches = math.ceil(len(train_df) / batchSize)
    while True: 
        train_df = train_df.sample(frac=1.)   
        for i in range(numBatches):
            texts = train_df.iloc[i*batchSize:(i+1)*batchSize, 1]
            textArr = np.array([textToArr(text) for text in texts])
            yield textArr, np.array(train_df["target"][i*batchSize:(i+1)*batchSize])

In [0]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Dropout

Using TensorFlow backend.


In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred): 
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred): 
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(30, 300)))
#model.add(Dropout(0.2))
#model.add(Bidirectional(CuDNNLSTM(64)))
#model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])

In [0]:
bg = batchGenerate(trainDf)
model.fit_generator(bg, epochs=20, steps_per_epoch=1000, validation_data=(valVects, valY), verbose=True) 

NameError: ignored

In [0]:
score = model.evaluate(testVects, testY, batch_size=1024, verbose=1)

22570/22570 [==============================] - 2s 87us/step


In [0]:
print(model.metrics_names[0],":",score[0])
print(model.metrics_names[1],":",score[1])
print(model.metrics_names[2],":",score[2])

loss : 0.34119263452378307
acc : 0.949490474080638
f1 : 0.5371880054262607
